In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shakeelahmedshaik","key":"7ee0f9a75504fe8b80a31696738bf91c"}'}

In [50]:
import os

# Directory containing images
image_dir = 'food41/images'

# Function to calculate the dataset size
def calculate_dataset_size(image_dir):
    total_images = 0
    for category in os.listdir(image_dir):
        category_path = os.path.join(image_dir, category)
        if os.path.isdir(category_path):
            total_images += len(os.listdir(category_path))
    return total_images

# Calculate and print the dataset size
dataset_size = calculate_dataset_size(image_dir)
print(f'Total number of images in the dataset: {dataset_size}')


Total number of images in the dataset: 101000


In [46]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import os

# Define the model architecture as ResNet-50
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 1)

    def forward(self, x):
        return self.model(x)

# Define the function to load the model
def load_model(model_path, device):
    model = MyModel()
    state_dict = torch.load(model_path, map_location=device)
    if 'model' in state_dict:
        state_dict = state_dict['model']
    model.load_state_dict(state_dict, strict=False)
    model.to(device)
    model.eval()
    return model

# Define the preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the function to perform predictions
def predict(test_data, image_dir, model, transform, device):
    predicted_calories = []
    for img_path in test_data['image_path']:
        full_img_path = os.path.join(image_dir, img_path)
        if not os.path.exists(full_img_path):
            raise FileNotFoundError(f"Image not found: {full_img_path}")
        image = Image.open(full_img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(image)
            calories = output.item()
        predicted_calories.append(calories)
    return predicted_calories

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your model
model_path = '/content/drive/My Drive/ColabNotebooks/models/food_calorie_estimator.pth'
model = load_model(model_path, device)

# Directory containing images
image_dir = 'food41/images'

# Test data with actual image paths
test_data = pd.DataFrame({
    'image_path': [
        'apple_pie/3586158.jpg',
        'baklava/49917.jpg'
        # Add more paths as needed
    ]
})

# Perform predictions
predicted_calories = predict(test_data, image_dir, model, transform, device)
for img_path, calories in zip(test_data['image_path'], predicted_calories):
    print(f'Image: {img_path}, Predicted Calorie Content: {calories:.2f} kcal')


Image: apple_pie/3586158.jpg, Predicted Calorie Content: 0.16 kcal
Image: baklava/49917.jpg, Predicted Calorie Content: 0.05 kcal
